In [1]:
import time
import numpy as np
import pandas as pd
from sklearn import preprocessing, feature_extraction, model_selection
from sklearn.metrics import mean_squared_error, accuracy_score, plot_confusion_matrix, f1_score
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, confusion_matrix

import stellargraph as sg
from stellargraph import datasets
from stellargraph.mapper import (
    CorruptedGenerator,
    FullBatchNodeGenerator,
    GraphSAGENodeGenerator,
    HinSAGENodeGenerator,
    Node2VecNodeGenerator,
    ClusterNodeGenerator,
)
from stellargraph.layer import GCN, DeepGraphInfomax, GraphSAGE, GAT, APPNP, HinSAGE, Dense

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Model, optimizers, losses, metrics

from imblearn.over_sampling import SMOTE, RandomOverSampler, ADASYN, BorderlineSMOTE, KMeansSMOTE, SMOTENC, SVMSMOTE
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.metrics import classification_report_imbalanced

from collections import Counter, defaultdict
import multiprocessing
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-whitegrid')
%matplotlib inline

In [2]:
import sys
sys.path.append('../')
import utils

In [3]:
v_data, e_data, v_sets, e_sets, core_targets, ext_targets, core_testing = utils.load_for_jupyter()

Dataset already downloaded. Loading it from file system
LOADING DATA: 1.02 s
PREPROCESSING DATA: 387.63 s


In [4]:
st_sed = e_sets['is similar'].loc[e_sets['is similar'].weight == 1]

In [5]:
st_sed.shape

(36672, 3)

In [6]:
src = v_data[v_data.index.isin(st_sed.source.values)]
tgt = v_data[v_data.index.isin(st_sed.target.values)]

nodes = pd.concat([src, tgt])

In [7]:
st_sed

,source,target,weight
edge_id,,,
600111766073,100109641,3001157349,1.0
600111766438,100109660,3001151288,1.0
600111762366,100109664,3001153481,1.0
600111767992,100109673,3001175846,1.0
600111763004,100109676,3001180125,1.0
...,...,...,...
650111802554,20030017540,3001170561,1.0
650111779829,20030017541,3001165543,1.0
650111797101,20030017543,3001179268,1.0


In [8]:
# first source
v_data.loc[v_data.index == 100109641]

,Label,Revenue Size Flag,Account ID String,Address,Person or Organisation,Name,Income Size Flag,CoreCaseGraphID,ExtendedCaseGraphID,testingFlag
node_id,,,,,,,,,,
100109641,Customer,NaN,NaN,NaN,Person,VALEA GOODWILL FOUNDATION,high,2381.0,2381.0,0.0


In [13]:
# first target
v_data.loc[v_data.index == 3001157349]

,Label,Revenue Size Flag,Account ID String,Address,Person or Organisation,Name,Income Size Flag,CoreCaseGraphID,ExtendedCaseGraphID,testingFlag
node_id,,,,,,,,,,
3001157349,External Entity,NaN,NaN,NaN,Person,Hqhlm01YYc,NaN,NaN,2381.0,0.0


In [9]:
# second source
v_data.loc[v_data.index == 100109660]

,Label,Revenue Size Flag,Account ID String,Address,Person or Organisation,Name,Income Size Flag,CoreCaseGraphID,ExtendedCaseGraphID,testingFlag
node_id,,,,,,,,,,
100109660,Customer,NaN,NaN,NaN,Organisation,CHU MICHAEL H F,medium,NaN,2032.0,0.0


In [14]:
# second target
v_data.loc[v_data.index == 3001151288]

,Label,Revenue Size Flag,Account ID String,Address,Person or Organisation,Name,Income Size Flag,CoreCaseGraphID,ExtendedCaseGraphID,testingFlag
node_id,,,,,,,,,,
3001151288,External Entity,NaN,NaN,NaN,Organisation,dxa669qsKE,NaN,NaN,NaN,NaN


In [10]:
v_data.loc[v_data.ExtendedCaseGraphID == 2032]

,Label,Revenue Size Flag,Account ID String,Address,Person or Organisation,Name,Income Size Flag,CoreCaseGraphID,ExtendedCaseGraphID,testingFlag
node_id,,,,,,,,,,
15020048483,Account,mid_high,Yhi46S2waH,NaN,NaN,NaN,NaN,NaN,2032.0,0.0
15020075907,Account,mid_high,mg9AWpnb6L,NaN,NaN,NaN,NaN,NaN,2032.0,0.0
250113288,Address,NaN,NaN,gvN2MVSQnv,NaN,NaN,NaN,NaN,2032.0,0.0
100109660,Customer,NaN,NaN,NaN,Organisation,CHU MICHAEL H F,medium,NaN,2032.0,0.0
1001013097,Customer,NaN,NaN,NaN,Organisation,CHE HONG,high,NaN,2032.0,0.0
100102312,Customer,NaN,NaN,NaN,Person,PECKORY INC.,medium,NaN,2032.0,0.0
1001051857,Customer,NaN,NaN,NaN,Organisation,NPV6wSFz5w,medium,NaN,2032.0,0.0
1001035910,Customer,NaN,NaN,NaN,Person,56zZjSZLUp,low,NaN,2032.0,0.0
1001049076,Customer,NaN,NaN,NaN,Organisation,3cx1PmLu44,low,NaN,2032.0,0.0


In [11]:
core_targets.loc[core_targets.CaseID == 2032]

,CaseID
NodeID,
1001029510,2032
15020035193,2032
15020035195,2032
15020039077,2032


In [12]:
# Let's look at the first hop of the "is similar" connections

# Vorrei sapere quanti nodi che hanno connessioni di tipo "is similar" estendono il loro CaseID.
# Cioè quanti nodi presenti nel source e target hanno CoreCase e ExtendedCase ID.

In [50]:
v_datan = v_data.fillna(0)

In [100]:
sources_with_core_case = 0
sources_with_ext_case = 0
targets_with_core_case = 0
targets_with_ext_case = 0
sources = dict()
targets = dict()

i = 0
for ind, row in e_data.loc[e_data.Label == 'is similar'].iterrows():
    if v_datan.loc[v_datan.index == row.from_id].CoreCaseGraphID.values[0] != 0:
        sources_with_core_case += 1
        sources[i] = ind
    if v_datan.loc[v_datan.index == row.to_id].CoreCaseGraphID.values[0] != 0:
        targets_with_core_case += 1
        targets[i] = ind
    if v_datan.loc[v_datan.index == row.from_id].ExtendedCaseGraphID.values[0] != 0:
        sources_with_ext_case += 1
    if v_datan.loc[v_datan.index == row.to_id].ExtendedCaseGraphID.values[0] != 0:
        targets_with_ext_case += 1
    i +=1
        

KeyboardInterrupt: 

In [77]:
sources_with_core_case

7343

In [78]:
targets_with_core_case

2005

In [79]:
sources_with_ext_case

21015

In [80]:
targets_with_ext_case

12929

In [83]:
# Sembra che molte connessioni in cui il source ha un case ID abbiano il target con lo stesso Case ID.
# Con che accuracy succede questa cosa?

In [101]:
# try with the strong similarities

for ind, row in e_data.loc[e_data.Label == 'is similar'].iterrows():
    x = v_datan.loc[v_datan.index == row.from_id].CoreCaseGraphID.values[0]
    x2 = v_datan.loc[v_datan.index == row.from_id].ExtendedCaseGraphID.values[0]
    if x != 0 or x2 != 0:
        y = v_datan.loc[v_datan.index == row.to_id].CoreCaseGraphID.values[0]
        y2 = v_datan.loc[v_datan.index == row.to_id].ExtendedCaseGraphID.values[0]
        print(f"Edge {ind} has source node {row.from_id} and target node {row.to_id}.")
        print(f"\t sourceCoreID:{x} - sourceExtID:{x2} - targetCoreID:{y} - targetExtID:{y2}")
    

Edge 600111768422 has source node 100109621 and target node 3001181933.
	 sourceCoreID:0.0 - sourceExtID:538.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 600111766073 has source node 100109641 and target node 3001157349.
	 sourceCoreID:2381.0 - sourceExtID:2381.0 - targetCoreID:0.0 - targetExtID:2381.0
Edge 600111764661 has source node 100109650 and target node 3001163177.
	 sourceCoreID:0.0 - sourceExtID:439.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 600111766438 has source node 100109660 and target node 3001151288.
	 sourceCoreID:0.0 - sourceExtID:2032.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 600111769197 has source node 100109678 and target node 3001141779.
	 sourceCoreID:0.0 - sourceExtID:1025.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 600111767504 has source node 100109696 and target node 3001171957.
	 sourceCoreID:639.0 - sourceExtID:0.0 - targetCoreID:639.0 - targetExtID:0.0
Edge 600111768002 has source node 100109708 and target node 3001155694.
	 sourceCoreID:0.0 - sourceEx

Edge 600111769332 has source node 1001013219 and target node 3001183026.
	 sourceCoreID:1866.0 - sourceExtID:0.0 - targetCoreID:0.0 - targetExtID:1866.0
Edge 600111767377 has source node 1001013268 and target node 3001146195.
	 sourceCoreID:0.0 - sourceExtID:2560.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 600111761568 has source node 1001013340 and target node 3001175455.
	 sourceCoreID:0.0 - sourceExtID:358.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 600111763162 has source node 1001013350 and target node 3001184352.
	 sourceCoreID:254.0 - sourceExtID:0.0 - targetCoreID:0.0 - targetExtID:254.0
Edge 600111764423 has source node 1001013373 and target node 3001173616.
	 sourceCoreID:0.0 - sourceExtID:1029.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 600111762957 has source node 1001013393 and target node 3001179093.
	 sourceCoreID:2043.0 - sourceExtID:2043.0 - targetCoreID:2043.0 - targetExtID:2043.0
Edge 600111767038 has source node 1001013446 and target node 3001186528.
	 sourceCoreID:

Edge 600111769808 has source node 1001016540 and target node 3001182070.
	 sourceCoreID:0.0 - sourceExtID:3648.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 600111768460 has source node 1001016576 and target node 3001173213.
	 sourceCoreID:0.0 - sourceExtID:926.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 600111765244 has source node 1001016592 and target node 3001167882.
	 sourceCoreID:0.0 - sourceExtID:1394.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 600111766355 has source node 1001016626 and target node 3001159941.
	 sourceCoreID:0.0 - sourceExtID:2780.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 600111763550 has source node 1001016633 and target node 3001162717.
	 sourceCoreID:160.0 - sourceExtID:0.0 - targetCoreID:0.0 - targetExtID:160.0
Edge 600111768617 has source node 1001016659 and target node 3001154176.
	 sourceCoreID:0.0 - sourceExtID:236.0 - targetCoreID:0.0 - targetExtID:3085.0
Edge 600111764853 has source node 1001016659 and target node 3001187786.
	 sourceCoreID:0.0 - sour

Edge 600111765900 has source node 1001020402 and target node 3001171651.
	 sourceCoreID:0.0 - sourceExtID:1106.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 600111768516 has source node 1001020449 and target node 3001135726.
	 sourceCoreID:0.0 - sourceExtID:230.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 600111766129 has source node 1001020449 and target node 3001169948.
	 sourceCoreID:0.0 - sourceExtID:230.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 600111769227 has source node 1001020562 and target node 3001145519.
	 sourceCoreID:0.0 - sourceExtID:1419.0 - targetCoreID:0.0 - targetExtID:3229.0
Edge 600111760565 has source node 1001020592 and target node 3001157260.
	 sourceCoreID:1982.0 - sourceExtID:1982.0 - targetCoreID:1982.0 - targetExtID:1982.0
Edge 600111761287 has source node 1001020595 and target node 3001165097.
	 sourceCoreID:0.0 - sourceExtID:3653.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 600111763754 has source node 1001020610 and target node 3001149209.
	 sourceCoreID:0.

Edge 600111760113 has source node 1001022821 and target node 3001132908.
	 sourceCoreID:0.0 - sourceExtID:1121.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 600111761039 has source node 1001022938 and target node 3001174400.
	 sourceCoreID:0.0 - sourceExtID:156.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 600111765509 has source node 1001022957 and target node 3001147614.
	 sourceCoreID:0.0 - sourceExtID:1313.0 - targetCoreID:0.0 - targetExtID:1313.0
Edge 600111761388 has source node 1001022972 and target node 3001171128.
	 sourceCoreID:0.0 - sourceExtID:22.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 600111762416 has source node 1001023007 and target node 3001157480.
	 sourceCoreID:0.0 - sourceExtID:861.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 600111765886 has source node 1001023086 and target node 3001138948.
	 sourceCoreID:0.0 - sourceExtID:1408.0 - targetCoreID:0.0 - targetExtID:3221.0
Edge 600111762213 has source node 1001023110 and target node 3001168560.
	 sourceCoreID:0.0 - sour

...

Edge 650111858466 has source node 20030013138 and target node 3001147170.
	 sourceCoreID:3759.0 - sourceExtID:3759.0 - targetCoreID:0.0 - targetExtID:3759.0
Edge 650111860637 has source node 20030013138 and target node 3001171310.
	 sourceCoreID:3759.0 - sourceExtID:3759.0 - targetCoreID:0.0 - targetExtID:3759.0
Edge 650111854609 has source node 20030013140 and target node 3001183249.
	 sourceCoreID:0.0 - sourceExtID:892.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 650111847308 has source node 20030013140 and target node 3001140524.
	 sourceCoreID:0.0 - sourceExtID:892.0 - targetCoreID:2996.0 - targetExtID:2996.0
Edge 650111847772 has source node 20030013140 and target node 3001146888.
	 sourceCoreID:0.0 - sourceExtID:892.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 650111771390 has source node 20030013140 and target node 3001147888.
	 sourceCoreID:0.0 - sourceExtID:892.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 650111772068 has source node 20030013140 and target node 3001169226.
	 sour

Edge 650111790780 has source node 20030013406 and target node 3001159955.
	 sourceCoreID:0.0 - sourceExtID:2663.0 - targetCoreID:0.0 - targetExtID:2663.0
Edge 650111831739 has source node 20030013406 and target node 3001135552.
	 sourceCoreID:0.0 - sourceExtID:2663.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 650111854295 has source node 20030013406 and target node 3001170846.
	 sourceCoreID:0.0 - sourceExtID:2663.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 650111792550 has source node 20030013406 and target node 3001177014.
	 sourceCoreID:0.0 - sourceExtID:2663.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 650111805172 has source node 20030013423 and target node 3001182256.
	 sourceCoreID:0.0 - sourceExtID:3882.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 650111798182 has source node 20030013423 and target node 3001136758.
	 sourceCoreID:0.0 - sourceExtID:3882.0 - targetCoreID:0.0 - targetExtID:3882.0
Edge 650111842930 has source node 20030013423 and target node 3001158249.
	 sourceCoreID

Edge 650111821751 has source node 20030017031 and target node 3001131554.
	 sourceCoreID:0.0 - sourceExtID:122.0 - targetCoreID:0.0 - targetExtID:122.0
Edge 650111826998 has source node 20030017031 and target node 3001153508.
	 sourceCoreID:0.0 - sourceExtID:122.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 650111830993 has source node 20030017031 and target node 3001154367.
	 sourceCoreID:0.0 - sourceExtID:122.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 650111803796 has source node 20030017031 and target node 3001170666.
	 sourceCoreID:0.0 - sourceExtID:122.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 650111853033 has source node 20030017031 and target node 3001184399.
	 sourceCoreID:0.0 - sourceExtID:122.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 650111841923 has source node 20030017031 and target node 3001141596.
	 sourceCoreID:0.0 - sourceExtID:122.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 650111861955 has source node 20030017031 and target node 3001175929.
	 sourceCoreID:0.0 - sou

Edge 650111774326 has source node 20030017179 and target node 3001134190.
	 sourceCoreID:0.0 - sourceExtID:1688.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 650111834709 has source node 20030017179 and target node 3001182106.
	 sourceCoreID:0.0 - sourceExtID:1688.0 - targetCoreID:0.0 - targetExtID:1688.0
Edge 650111849098 has source node 20030017179 and target node 3001158368.
	 sourceCoreID:0.0 - sourceExtID:1688.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 650111806981 has source node 20030017179 and target node 3001158751.
	 sourceCoreID:0.0 - sourceExtID:1688.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 650111815764 has source node 20030017203 and target node 3001133182.
	 sourceCoreID:497.0 - sourceExtID:497.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 650111834339 has source node 20030017203 and target node 3001170607.
	 sourceCoreID:497.0 - sourceExtID:497.0 - targetCoreID:0.0 - targetExtID:497.0
Edge 650111835815 has source node 20030017203 and target node 3001139556.
	 sourceCoreI

Edge 650111817300 has source node 20030017368 and target node 3001170524.
	 sourceCoreID:0.0 - sourceExtID:1977.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 650111840356 has source node 20030017368 and target node 3001134965.
	 sourceCoreID:0.0 - sourceExtID:1977.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 650111839887 has source node 20030017368 and target node 3001136323.
	 sourceCoreID:0.0 - sourceExtID:1977.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 650111839645 has source node 20030017368 and target node 3001183285.
	 sourceCoreID:0.0 - sourceExtID:1977.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 650111837321 has source node 20030017368 and target node 3001145426.
	 sourceCoreID:0.0 - sourceExtID:1977.0 - targetCoreID:0.0 - targetExtID:0.0
Edge 650111841619 has source node 20030017368 and target node 3001162703.
	 sourceCoreID:0.0 - sourceExtID:1977.0 - targetCoreID:0.0 - targetExtID:1977.0
Edge 650111825646 has source node 20030017368 and target node 3001158937.
	 sourceCoreID:0.

In [103]:
G = sg.StellarDiGraph(v_sets, e_sets)

ValueError: expected IDs to appear once, found some that appeared more: 150200739, 150200774, 1502004106, 1502004149, 1502004436, 1502004892, 1502005084, 1502005262, 1502005557, 1502007954, 1502008378, 1502008500, 1502009330, 15020010877, 15020010969, 15020012238, 15020012893, 15020015232, 15020018029, 15020022245, ... (11 more)